In [ ]:
import os
import sys
import json
import pandas as pd
import numpy as np
import mwparserfromhell as mw
import seaborn as sns
import matplotlib.pyplot as plt
import re
sns.set(style="whitegrid")
%matplotlib inline

%load_ext autoreload
%autoreload 2
import processing # this makes the "processing" folder available as a python module

from features.strengths import get_strengths, get_features
from utils import remove_ref, get_templates

In [ ]:
battles = json.load(open("../datasets/battle-fields-0.json"))
df = pd.DataFrame([b["infobox"] for b in battles if not b["infobox"].get("error")])
pattern = re.compile("^strength[0-9]")
df = df[[b for b in df.columns if pattern.match(b)]]

In [ ]:
missed1 = get_strengths(df, df['strength1'],'1')

In [ ]:
missed2 = get_strengths(df, df['strength2'],'2')

In [ ]:
missed3 = get_strengths(df, df['strength3'],'3')

In [ ]:
df.to_csv("stengths")

In [ ]:
print("For each strength column we have ", missed1, " ", missed2, " ", missed3, " lines that contain a number but are not parsed.")

Which means that almost 100% of the values have been parsed in each column:

In [ ]:
toPlot = {'strength1': 100-100*missed1/len(df), 'strength2': 100-(100*missed2/len(df)), 'strength3':100-(100*missed3/len(df))}

plt.bar(range(len(toPlot)), toPlot.values())
plt.title("Percent of parsed rows")
plt.xticks(range(len(toPlot)), toPlot.keys())
plt.show()

In [ ]:
pd.options.display.max_colwidth = 20
df[['strength1', 'strength_1', 'strength2', 'strength_2', 'strength3', 'strength_3']].head(10)

But, we also observe that for a lot of battles, the strengths are not given numerically:

In [ ]:
print("Out of ", len(df), " battles")
df = df.replace(0, np.NaN)
print("Strength1 has ", df['strength_1'].isnull().sum(), "null values")
print("Strength2 has ", df['strength_2'].isnull().sum(), "null values")
print("Strength3 has ", df['strength_3'].isnull().sum(), "null values")

toPlot = {'strength1': 100-100*df['strength_1'].isnull().sum()/len(df), 'strength2': 100-(100*df['strength_2'].isnull().sum()/len(df)), 'strength3':100-(100*df['strength_3'].isnull().sum()/len(df))}

plt.bar(range(len(toPlot)), toPlot.values())
plt.title("Percent of battles with numeric values for strength")
plt.xticks(range(len(toPlot)), toPlot.keys())
plt.show()

We also observe that almost 70% of the battles have numeric values for two combatants.

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(111)

ax1.scatter(range(len(df)), df['strength_1'], c='b', marker="s", label='strength_1')
ax1.scatter(range(len(df)), df['strength_2'], c='r', marker="o", label='strength_2')
plt.title("Comparison between strengths of different combatants")
plt.legend(loc='upper left');
plt.show()

In [ ]:
cbt_1 = np.array(df["combatant_1"].dropna())
cbt_2 = np.array(df["combatant_2"].dropna())
cbt_3 = np.array(df["combatant_3"].dropna())
print(len(cbt_1), len(cbt_2), len(cbt_3))

cbt_all = np.concatenate((cbt_1, cbt_2, cbt_3))

all_cbt_names = [c for cl in cbt_all for c in cl]
names = pd.Series(all_cbt_names)
print(len(set(all_cbt_names)))

f, ax = plt.subplots(figsize=(6, 15))
counts = names.value_counts().sort_values(ascending=False)
counts = counts.head(50)
sns.barplot(x=counts, y=counts.index, ax=ax)